In [ ]:
# This experiment is based on https://github.com/benmoseley/harmonic-oscillator-pinn-workshop
# Authored by B. Moseley
# under MIT license

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import tqdm
import os

Settings

---

In [ ]:
lr = 1e-4
d_hidden = 32
l = 1e-4
plot_freq = 5000
n_iter = 30000
n_points = 100

Configure the ground truth

---

In [ ]:
#Set the initial conditions
x_0 = 1.0
v_0 = 0.0

#Set the parameters of the system
m = 1.0
w_0 = 20
delta = 2

mu = delta * 2 * m
k = m * w_0**2
w = np.sqrt(w_0**2 - delta**2)
phi = np.arctan(-delta / w)
A = 1 / (2 * np.cos(phi))
print(phi)

print("m = ",m)
print("k = ",k)
print("mu = ",mu)

Generate training data

---

In [ ]:
def solution(t):
    y = np.exp(-delta * t) * (2 * A * torch.cos(phi + w * t))
    return y

t_data = torch.linspace(0, 0.401, 20).reshape(-1,1)
x_data = solution(t_data).reshape(-1,1)

In [ ]:
t_eval = torch.linspace(0, 1,1000)
plt.plot(t_eval, solution(t_eval))
plt.scatter(t_data.squeeze(), x_data.squeeze(), color='red')

Train a model with the PINN loss

---

In [ ]:
u_PI = nn.Sequential(
    nn.Linear(1, d_hidden),
    nn.Tanh(),
    nn.Linear(d_hidden, d_hidden),
    nn.Tanh(),
    nn.Linear(d_hidden, d_hidden),
    nn.Tanh(),
    nn.Linear(d_hidden, d_hidden),
    nn.Tanh(),
    nn.Linear(d_hidden, 1)
)

optimizer = torch.optim.Adam(u_PI.parameters(), lr=lr)

t = torch.linspace(0, 1.0, n_points).reshape(-1, 1).requires_grad_(True)

for i in tqdm.tqdm(range(n_iter)):

    x = u_PI(t)
    x_dot = torch.autograd.grad(x.sum(), t, create_graph=True,retain_graph=True)[0]
    x_dot_dot = torch.autograd.grad(x_dot.sum(), t, create_graph=True,retain_graph=True)[0]

    r = (m * x_dot_dot + mu *  x_dot + k *  x).squeeze().pow(2)
    loss_r = r.mean()

    loss_data = (u_PI(t_data) - x_data).pow(2).squeeze().mean()
   
    loss =  loss_data + l * loss_r 

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

train a model without PINN loss:
---

In [ ]:
u_plain = nn.Sequential(
    nn.Linear(1, d_hidden),
    nn.Tanh(),
    nn.Linear(d_hidden, d_hidden),
    nn.Tanh(),
    nn.Linear(d_hidden, d_hidden),
    nn.Tanh(),
    nn.Linear(d_hidden, d_hidden),
    nn.Tanh(),
    nn.Linear(d_hidden, 1)
)

optimizer = torch.optim.Adam(u_plain.parameters(), lr=lr)

t = torch.linspace(0, 1.0, n_points).reshape(-1, 1).requires_grad_(True)

for i in tqdm.tqdm(range(n_iter)):

    loss_data = (u_plain(t_data) - x_data).pow(2).squeeze().mean()
   
    loss =  loss_data 

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Compare the results

---

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(12,5))

fs = 20
t_eval = torch.linspace(0, 1,1000)
ax.plot(t_eval, solution(t_eval),color = "k",ls = "dashed",label = "true solution",lw = 3)
ax.scatter(t_data.squeeze(), x_data.squeeze(), color='red',label = "training data",s = 100)

ax.plot(t_eval.detach().numpy(), u_PI(t_eval.reshape(-1,1)).detach().squeeze().numpy(), color='green',label = "data + PI loss",lw = 3)
ax.plot(t_eval.detach().numpy(), u_plain(t_eval.reshape(-1,1)).detach().squeeze().numpy(), color='blue',label = "data",lw = 3)
ax.tick_params(axis='both', which='major', labelsize=fs)

ax.set_xlabel("t",fontsize = fs)
ax.set_ylabel("u(t)",fontsize = fs)

plt.tight_layout()

handles, labels = [], []

for handle, label in zip(*ax.get_legend_handles_labels()):
    handles.append(handle)
    labels.append(label)

# Add a single legend below all subplots
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.05), ncol=4,fontsize = fs)

os.makedirs("../../results/harmonic_oscillator/")
plt.savefig(
    "../../results/harmonic_oscillator/PINN_damped_oscillator.pdf",
    bbox_inches='tight'
    )
plt.close(fig)